In [1]:
#export
"""
This is for functions that sort of changes the table
structure in a dramatic way. They're the core transformations
"""
from typing import List, Union, Iterator, Callable, Any, Tuple, Dict
from collections import defaultdict, Counter
from k1lib.bioinfo.cli.init import patchDefaultDelim, BaseCli, oneToMany, T, Table
import k1lib.bioinfo.cli as cli
import itertools, numpy as np, torch
__all__ = ["joinColumns", "transpose", "splitColumns", "joinList", "splitList",
           "joinStreams", "joinStreamsRandom", "batched", "collate",
           "insertRow", "insertColumn", "insertIdColumn",
           "toDict", "split", "expandE", "table", "stitch",
           "listToTable", "tableFromList",
           "count", "permute", "accumulate", "AA_", "peek", "peekF",
           "repeat", "infiniteFrom"]

In [2]:
#export
class joinColumns(BaseCli):
    def __init__(self, fillValue=None):
        """Join multiple columns and loop through all rows. Aka transpose.

:param fillValue: if not None, then will try to zip longest with this fill value

Example::

    # returns [[1, 4], [2, 5], [3, 6]]
    [[1, 2, 3], [4, 5, 6]] | joinColumns() | dereference()
    # returns [[1, 4], [2, 5], [3, 6], [0, 7]]
    [[1, 2, 3], [4, 5, 6, 7]] | joinColumns(0) | dereference()"""
        super().__init__(); self.fillValue = fillValue
    def __ror__(self, it:Iterator[Iterator[T]]) -> Table[T]:
        super().__ror__(it)
        if self.fillValue is None: yield from zip(*it)
        else: yield from itertools.zip_longest(*it, fillvalue=self.fillValue)
splitColumns = transpose = joinColumns

In [3]:
assert [[1, 2, 3], [4, 5, 6]] | joinColumns() | cli.dereference() == [[1, 4], [2, 5], [3, 6]]
assert [[1, 2, 3], [4, 5, 6, 7]] | joinColumns(0) | cli.dereference() == [[1, 4], [2, 5], [3, 6], [0, 7]]

In [4]:
#export
class joinList(BaseCli):
    def __init__(self, element=None, begin=True):
        """Join element into list.

:param element: the element to insert. If None, then takes the input [e, [...]],
    else takes the input [...] as usual

Example::

    # returns [5, 2, 6, 8]
    [5, [2, 6, 8]] | joinList()
    # also returns [5, 2, 6, 8]
    [2, 6, 8] | joinList(5)"""
        super().__init__(); self.element = element; self.begin = begin
    def __ror__(self, it:Tuple[T, Iterator[T]]) -> Iterator[T]:
        super().__ror__(it); it = iter(it)
        if self.element is None:
            if self.begin: yield next(it); yield from next(it)
            else: e = next(it); yield from next(it); yield e
        else:
            if self.begin: yield self.element; yield from it
            else: yield from it; yield self.element

In [5]:
assert [5, [2, 6, 8]] | joinList() | cli.dereference() == [5, 2, 6, 8]
assert [2, 6, 8] | joinList(5) | cli.dereference() == [5, 2, 6, 8]

In [6]:
#export
class splitList(BaseCli):
    def __init__(self, weights:List[float]=[0.8, 0.2]):
        """Splits list of elements into multiple lists.
Example::

    # returns [[0, 1, 2, 3, 4, 5, 6, 7], [8, 9]]
    range(10) | splitList([0.8, 0.2]) | dereference()"""
        super().__init__(); self.weights = np.array(weights)
    def __ror__(self, it):
        super().__ror__(it); it = list(it); ws = self.weights; c = 0
        ws = (ws * len(it) / ws.sum()).astype(int)
        for w in ws: yield it[c:c+w]; c += w

In [7]:
assert range(100) | splitList() | cli.dereference() == [range(80), range(80, 100)] | cli.dereference()
assert range(10) | splitList() | cli.dereference() == [[0, 1, 2, 3, 4, 5, 6, 7], [8, 9]]

In [8]:
#export
class joinStreams(BaseCli):
    """Join multiple streams.
Example::

    # returns [1, 2, 3, 4, 5]
    [[1, 2, 3], [4, 5]] | joinStreams() | dereference()"""
    def __ror__(self, streams:Iterator[Iterator[T]]) -> Iterator[T]:
        super().__ror__(streams)
        for stream in streams: yield from stream
import random
def rand(n):
    while True: yield random.randrange(n)
class joinStreamsRandom(BaseCli):
    """Join multiple streams randomly. If any streams runs out, then quits.
Example::

    # could return [0, 1, 10, 2, 11, 12, 13, ...], with max length 20, typical length 18
    [range(0, 10), range(10, 20)] | joinStreamsRandom() | dereference()"""
    def __ror__(self, streams:Iterator[Iterator[T]]) -> Iterator[T]:
        super().__ror__(streams)
        streams = [iter(st) for st in streams]
        try:
            for streamIdx in rand(len(streams)):
                yield next(streams[streamIdx])
        except StopIteration: pass

In [9]:
assert [[1, 2, 3], [4, 5]] | joinStreams() | cli.dereference() == list(range(1, 6))
assert len([range(0, 10), range(10, 20)] | joinStreamsRandom() | cli.dereference()) < 20

In [43]:
#export
class batched(BaseCli):
    def __init__(self, bs=32, includeLast=False):
        """Batches the input stream.
Example::

    # returns [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    range(11) | batched(3) | dereference()
    # returns [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10]]
    range(11) | batched(3, True) | dereference()
    # returns [[0, 1, 2, 3, 4]]
    range(5) | batched(float("inf"), True) | dereference()
    # returns []
    range(5) | batched(float("inf"), False) | dereference()"""
        super().__init__(); self.bs = bs; self.includeLast = includeLast
    def __ror__(self, it):
        super().__ror__(it); it = iter(it)
        l = []
        if self.bs == float("inf"):
            if self.includeLast: yield list(it)
            else: pass
            return
        try:
            while True:
                for i in range(self.bs): l.append(next(it))
                yield l; l = []
        except StopIteration:
            if self.includeLast: yield l

In [46]:
assert range(11) | batched(3) | cli.dereference() == [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
assert range(11) | batched(3, True) | cli.dereference() == [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10]]
assert range(5) | batched(float("inf"), True) | cli.dereference() == [[0, 1, 2, 3, 4]]
assert range(5) | batched(float("inf"), False) | cli.dereference() == []

In [12]:
#export
def collate():
    """Puts individual columns into a tensor.
Example::

    # returns [tensor([ 0, 10, 20]), tensor([ 1, 11, 21]), tensor([ 2, 12, 22])]
    [range(0, 3), range(10, 13), range(20, 23)] | collate() | toList()"""
    return transpose() | cli.apply(lambda row: torch.tensor(row))

In [13]:
a = [[range(0, 3), range(10, 13), range(20, 23)] | collate() | cli.toList(),
 [torch.tensor([ 0, 10, 20]), torch.tensor([ 1, 11, 21]), torch.tensor([ 2, 12, 22])]]
assert all(a | cli.transpose() | cli.apply(lambda ts: all(ts[0] == ts[1])) | cli.dereference())

In [14]:
#export
def insertRow(*row:List[T]):
    """Inserts a row right before every other rows. See also: :meth:`joinList`."""
    return joinList(row)
def insertColumn(*column, begin=True, fillValue=""):
    """Inserts a column at beginning or end.
Example::

    # returns [['a', 1, 2], ['b', 3, 4]]
    [[1, 2], [3, 4]] | insertColumn("a", "b") | dereference()
"""
    return transpose(fillValue) | joinList(column, begin) | transpose(fillValue)

In [15]:
assert [[1, 2, 3], ["a", "b", "c"]] | insertRow(2, 3, 4) | cli.dereference() == [[2, 3, 4], [1, 2, 3], ['a', 'b', 'c']]
assert [[1, 2], [3, 4]] | insertColumn("a", "b") | cli.dereference() == [['a', 1, 2], ['b', 3, 4]]

In [16]:
#export
def insertIdColumn(table=False, begin=True, fillValue=""):
    """Inserts an id column at the beginning (or end).
Example::

    # returns [[0, 'a', 2], [1, 'b', 4]]
    [["a", 2], ["b", 4]] | insertIdColumn(True) | dereference()
    # returns [[0, 'a'], [1, 'b']]
    "ab" | insertIdColumn()

:param table: if False, then insert column to an Iterator[str], else treat
    input as a full fledged table"""
    f = (cli.toRange() & transpose(fillValue)) | joinList(begin=begin) | transpose(fillValue)
    if table: return f
    else: return cli.wrapList() | transpose() | f

In [17]:
assert [["a", 2], ["b", 4]] | insertIdColumn(True) | cli.dereference() == [[0, 'a', 2], [1, 'b', 4]]
assert "ab" | insertIdColumn() | cli.dereference() == [[0, 'a'], [1, 'b']]

In [18]:
#export
class toDict(BaseCli):
    def __init__(self, keyF:Callable[[Any], str]=None, valueF:Callable[[Any], Any]=None):
        """Transform an incoming stream into a dict using a function for
values. Example::

    names = ["wanda", "vision", "loki", "mobius"]
    names | toDict(valueF=lambda s: len(s)) # will return {"wanda": 5, "vision": 6, ...}
    names | toDict(lambda s: s.title(), lambda s: len(s)) # will return {"Wanda": 5, "Vision": 6, ...}
"""
        super().__init__(); self.keyF = keyF or (lambda s: s)
        self.valueF = valueF or (lambda s: s)
    def __ror__(self, keys:Iterator[Any]) -> Dict[Any, Any]:
        super().__ror__(keys); keyF = self.keyF; valueF = self.valueF
        return {keyF(key):valueF(key) for key in keys}

In [19]:
assert ["wanda", "vision", "loki", "mobius"] | toDict(valueF=lambda s: len(s)) == {"wanda": 5, "vision": 6, "loki": 4, "mobius": 6}

In [20]:
#export
class split(BaseCli):
    def __init__(self, delim:str=None, idx:int=None):
        """Splits each line using a delimiter, and outputs the
parts as a separate line. Example::

    # returns ["a", "b", "d", "e"]
    ["a,b", "d,e"] | split(",") | dereference()
    # returns ['b', 'e']
    ["a,b", "d,e"] | split(",", 1) | dereference()

:param idx: if available, only outputs the element at that index"""
        super().__init__()
        self.delim = patchDefaultDelim(delim); self.idx = idx
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it)
        if self.idx == None:
            for line in it:
                for elem in line.split(self.delim): yield elem
        else:
            for line in it:
                elems = line.split(self.delim)
                yield elems[self.idx] if self.idx < len(elems) else None

In [21]:
assert ["a,b", "d,e"] | split(",") | cli.dereference() == ['a', 'b', 'd', 'e']
assert ["a,b", "d,e"] | split(",", 1) | cli.dereference() == ['b', 'e']

In [22]:
#export
class expandE(BaseCli):
    def __init__(self, f:Callable[[T], List[T]], column:int):
        """Expands table element to multiple columns.
Example::

    # returns [['abc', 3, -2], ['de', 2, -5]]
    [["abc", -2], ["de", -5]] | expandE(lambda e: (e, len(e)), 0) | dereference()

:param f: Function that transforms 1 row element to multiple
"""
        self.f = f; self.column = column
    def __ror__(self, it):
        f = self.f; c = self.column
        def gen(row):
            for i, e in enumerate(row):
                if i == c: yield from f(e)
                else: yield e
        return (gen(row) for row in it)

In [23]:
assert [["abc", -2], ["de", -5]] | expandE(lambda e: (e, len(e)), 0) | cli.dereference() == [['abc', 3, -2], ['de', 2, -5]]

In [24]:
#export
class table(BaseCli):
    def __init__(self, delim:str=None):
        """Splits lines to rows (List[str]) using a delimiter.
Example::

    # returns [['a', 'bd'], ['1', '2', '3']]
    ["a|bd", "1|2|3"] | table("|") | dereference()"""
        super().__init__(); self.delim = patchDefaultDelim(delim)
    def __ror__(self, it:Iterator[str]) -> Table[str]:
        super().__ror__(it)
        return (line.split(self.delim) for line in it)
class stitch(BaseCli):
    def __init__(self, delim:str=None):
        """Stitches elements in a row together, so they become a simple string.
See also: :class:`~k1lib.bioinfo.cli.output.pretty`. Example::

    # returns ['1|2', '3|4', '5|6']
    [[1, 2], [3, 4], [5, 6]] | stitch("|") | dereference()"""
        super().__init__(); self.delim = patchDefaultDelim(delim)
    def __ror__(self, it:Table[str]) -> Iterator[str]:
        super().__ror__(it); d = self.delim
        for row in it: yield d.join([str(e) for e in row])
def listToTable():
    """Turns Iterator[T] into Table[T]"""
    return cli.wrapList() | transpose()
tableFromList = listToTable

In [25]:
assert ["a|bd", "1|2|3"] | table("|") | cli.dereference() == [['a', 'bd'], ['1', '2', '3']]
assert [[1, 2], [3, 4], [5, 6]] | stitch("|") | cli.dereference() == ['1|2', '3|4', '5|6']

In [26]:
#export
class count(BaseCli):
    """Finds unique elements and returns a table with [frequency, value, percent]
columns. Example::

    # returns [[1, 'a', '33%'], [2, 'b', '67%']]
    ['a', 'b', 'b'] | count() | dereference()
"""
    def __ror__(self, it:Iterator[str]):
        it = it | cli.apply(lambda row: (tuple(row) if isinstance(row, list) else row))
        c = Counter(it); s = sum(c.values())
        for k, v in c.items(): yield [v, k, f"{round(100*v/s)}%"]

In [27]:
assert ['a', 'b', 'b'] | count() | cli.dereference() == [[1, 'a', '33%'], [2, 'b', '67%']]

In [28]:
#export
class permute(BaseCli):
    def __init__(self, *permutations:List[int]):
        """Permutes the columns. Acts kinda like :meth:`torch.Tensor.permute`.
Example::

    # returns [['b', 'a'], ['d', 'c']]
    ["ab", "cd"] | permute(1, 0) | dereference()
"""
        super().__init__(); self.permutations = permutations
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it); p = self.permutations
        def gen(row): row = list(row); return (row[i] for i in p)
        for row in it: yield gen(row)

In [29]:
assert ["ab", "cd"] | permute(1, 0) | cli.dereference() == [['b', 'a'], ['d', 'c']]
assert ["ab", "cd"] | permute(1, 0) | transpose() | cli.dereference() == [['b', 'd'], ['a', 'c']]

In [30]:
#export
class accumulate(BaseCli):
    def __init__(self, columnIdx:int=0, avg=False):
        """Groups lines that have the same row[columnIdx], and
add together all other columns, assuming they're numbers

:param columnIdx: common column index to accumulate
:param avg: calculate average values instead of sum"""
        super().__init__(); self.columnIdx = columnIdx; self.avg = avg
        self.dict = defaultdict(lambda: defaultdict(lambda: 0))
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it)
        for row in it:
            row = list(row); key = row[self.columnIdx]
            row.pop(self.columnIdx)
            for i, e in enumerate(row):
                try: self.dict[key][i] += float(e)
                except: self.dict[key][i] = e
        for key, values in self.dict.items():
            n = len(self.dict[key].keys())
            if self.avg:
                for i in range(n):
                    if isinstance(self.dict[key][i], (int, float)):
                        self.dict[key][i] /= n
            elems = [str(self.dict[key][i]) for i in range(n)]
            elems.insert(self.columnIdx, key)
            yield elems

In [31]:
#export
class AA_(BaseCli):
    def __init__(self, *idxs:List[int], wraps=False):
        """Returns 2 streams, one that has the selected element, and the other
the rest. Example::

    [1, 5, 6, 3, 7] | AA_(1) # will return [5, [1, 6, 3, 7]]

You can also put multiple indexes through::

    [1, 5, 6] | AA_(0, 2) # will return [[1, [5, 6]], [6, [1, 5]]]

If you put None in, then all indexes will be sliced::

    [1, 5, 6] | AA_(0, 2)

    # will return:
    # [[1, [5, 6]],
    #  [5, [1, 6]],
    #  [6, [1, 5]]]

As for why the strange name, think of this operation as "AĀ". In statistics,
say you have a set "A", then "not A" is commonly written as A with an overline
"Ā". So "AA\_" represents "AĀ", and that it first returns the selection A.

:param wraps: if True, then the first example will return [[5, [1, 6, 3, 7]]]
    instead, so that A has the same signature as Ā"""
        super().__init__(); self.idxs = idxs; self.wraps = wraps
        if len(idxs) == 0: raise AttributeError("Please enter some column indexes. If you wish to analyze all, put in `None`")
    def __ror__(self, it:List[Any]) -> List[List[List[Any]]]:
        super().__ror__(it); idxs = self.idxs; it = list(it)
        if len(idxs) == 1 and idxs[0] is None: idxs = range(len(it))
        def gen(idx):
            return [it[idx], [v for i, v in enumerate(it) if i != idx]]
        if not self.wraps and len(idxs) == 1: return gen(idxs[0])
        return [gen(idx) for idx in idxs]

In [32]:
#export
class peek(BaseCli):
    """Returns (firstRow, iterator). This sort of peaks at the first row, to
potentially gain some insights about the internal formats. Example::

    e, it = iter([[1, 2, 3], [1, 2]]) | peek()
    print(e) # prints "[1, 2, 3]"
    s = 0
    for e in it: s += len(e)
    print(s) # prints "5", or length of 2 lists"""
    def __ror__(self, it:Iterator[T]) -> Tuple[T, Iterator[T]]:
        super().__ror__(it)
        if (row := next(it, sentinel := object())) == sentinel: return None, []
        def gen(): yield row; yield from it
        return row, gen()

In [33]:
e, it = iter([[1, 2, 3], [1, 2]]) | peek(); assert e == [1, 2, 3]; s = 0
for e in it: s += len(e)
assert s == 5

In [34]:
#export
class peekF(BaseCli):
    def __init__(self, f:Union[BaseCli, Callable[[T], T]]):
        r"""Similar to :class:`peek`, but will execute ``f(row)`` and
return the input Iterator. Example::

    it = lambda: iter([[1, 2, 3], [1, 2]])
    # prints "[1, 2, 3]" and returns [[1, 2, 3], [1, 2]]
    it() | peekF(lambda x: print(x)) | dereference()
    # prints "1\n2\n3"
    it() | peekF(headOut()) | dereference()"""
        super().__init__(); self.f = f
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        super().__ror__(it)
        if (row := next(it, sentinel := object())) == sentinel: return []
        def gen(): yield row; yield from it
        self.f(row); return gen()

In [35]:
it = lambda: iter([[1, 2, 3], [1, 2]])
assert it() | peekF(lambda x: print(x)) | cli.dereference() == [[1, 2, 3], [1, 2]]
assert it() | peekF(cli.headOut()) | cli.dereference() == [[1, 2, 3], [1, 2]]

[1, 2, 3]
1
2
3


In [36]:
#export
class repeat(BaseCli):
    """Yields a specified amount of the passed in object.
Example::

    # returns [[1, 2, 3], [1, 2, 3], [1, 2, 3]]
    [1, 2, 3] | repeat(3) | toList()

:param repeat: if None, then repeats indefinitely"""
    def __init__(self, limit:int=None):
        super().__init__(); self.limit = limit
    def __ror__(self, o:T) -> Iterator[T]:
        super().__ror__(o)
        if self.limit is None:
            while True: yield o
        else:
            for _ in range(self.limit): yield o

In [37]:
assert [1, 2, 3] | repeat() | cli.head(3) | cli.dereference() == [[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [38]:
#export
class infiniteFrom(BaseCli):
    """Yields from a list. If runs out of elements, then do it again.
Example::

    # returns [1, 2, 3, 1, 2]
    [1, 2, 3] | infiniteFrom() | head(5) | dereference()"""
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        it = list(it)
        while True: yield from it

In [39]:
assert [1, 2, 3] | infiniteFrom() | cli.head(5) | cli.dereference() == [1, 2, 3, 1, 2]

In [48]:
!../../../export.py bioinfo/cli/structural

Current dir: /home/kelvin/repos/labs/k1lib, ../../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.12
Uninstalling k1lib-0.1.12:
  Successfully uninstalled k1lib-0.1.12
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1